# Flower Images Classification with TensorFlow 2.x on Cloud TPU:




### Weighted Averaging Ensemble from Models with Backbones: 


* EfficientNet B7 imagenet weights

* EfficientNet B6 imagenet weights

* EfficientNet B5 imagenet weights

* EfficientNet B4 imagenet weights





![](https://github.com/s-gladysh/flower-classification-with-tpus/raw/master/5.jpg)





The goal of this project is to investigate different deep neural network architectures, such as EfficientNet B7-B0 with ImageNet & noisy-student weights, DenseNet, ResNet, InceptionResNet, etc - to be applied as Backbone Networks in multi-class classification on Flower images dataset.

I explore the usage of Tensor Processing Units (TPU) in the Cloud, perform experiments with EfficientNet Hyper-parameters, Transfer Learning, Fine-Tuning, Image Augmentation and Weighted Averaging Ensembles created from various combinations of Backbone Networks.

Obtained results are evaluated and analyzed using Confusion Matrix, F-score, Precision and Recall.


*******************************








In [ ]:
#!pip install tf-nightly

In [ ]:
import tensorflow

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:
import numpy as np 
import pandas as pd 
import os

from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
import random, re, math, os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets

AUTO = tensorflow.data.experimental.AUTOTUNE

## Configuration and Data Access 


In [ ]:
IMAGE_SIZE = [512, 512] 


In [ ]:
try:
    tpu = tensorflow.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tensorflow.config.experimental_connect_to_cluster(tpu)
    tensorflow.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tensorflow.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tensorflow.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
BATCH_SIZE = 32 * strategy.num_replicas_in_sync

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')

GCS_PATH_SELECT = { 
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}

GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]




TRAINING_FILENAMES = tensorflow.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tensorflow.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tensorflow.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

In [ ]:

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

In [ ]:
def decode_image(image_data):
    image = tensorflow.image.decode_jpeg(image_data, channels=3)
    image = tensorflow.cast(image, tensorflow.float32) / 255.0  
    image = tensorflow.reshape(image, [*IMAGE_SIZE, 3]) 
    return image


def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tensorflow.io.FixedLenFeature([], tensorflow.string), 
        "class": tensorflow.io.FixedLenFeature([], tensorflow.int64),  
    }
    example = tensorflow.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tensorflow.cast(example['class'], tensorflow.int32)
    return image, label 


def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tensorflow.io.FixedLenFeature([], tensorflow.string), 
        "id": tensorflow.io.FixedLenFeature([], tensorflow.string), 
    }
    example = tensorflow.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum 


def load_dataset(filenames, labeled = True, ordered = False):
    
    ignore_order = tensorflow.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tensorflow.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) 
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset


def data_augment(image, label):

    image = tensorflow.image.random_flip_left_right(image)
    return image, label   

def get_training_dataset(dataset,do_aug=True):
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    if do_aug: dataset = dataset.map(transform, num_parallel_calls=AUTO)
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_validation_dataset(dataset):
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset


def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)


NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

## Data Augmentation



In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tensorflow.math.cos(rotation)
    s1 = tensorflow.math.sin(rotation)
    one = tensorflow.constant([1],dtype='float32')
    zero = tensorflow.constant([0],dtype='float32')
    rotation_matrix = tensorflow.reshape( tensorflow.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tensorflow.math.cos(shear)
    s2 = tensorflow.math.sin(shear)
    shear_matrix = tensorflow.reshape( tensorflow.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tensorflow.reshape( tensorflow.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tensorflow.reshape( tensorflow.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [ ]:
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tensorflow.random.normal([1],dtype='float32')
    shr = 5. * tensorflow.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tensorflow.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tensorflow.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tensorflow.random.normal([1],dtype='float32') 
    w_shift = 16. * tensorflow.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tensorflow.repeat( tensorflow.range(DIM//2,-DIM//2,-1), DIM )
    y = tensorflow.tile( tensorflow.range(-DIM//2,DIM//2),[DIM] )
    z = tensorflow.ones([DIM*DIM],dtype='int32')
    idx = tensorflow.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tensorflow.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tensorflow.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tensorflow.gather_nd(image,tensorflow.transpose(idx3))
        
    return tensorflow.reshape(d,[DIM,DIM,3]),label

In [ ]:
row = 3; col = 4;
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
one_element = tensorflow.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(transform).batch(row*col)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    plt.savefig("/kaggle/working/1.png")
    break

In [ ]:
row = 3; col = 4;
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
one_element = tensorflow.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(transform).batch(row*col)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
        plt.savefig("/kaggle/working/2.png")
    plt.show()
    break

In [ ]:
row = 3; col = 4;
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
one_element = tensorflow.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(transform).batch(row*col)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
        plt.savefig("/kaggle/working/3.png")
    plt.show()
    break

In [ ]:
row = 3; col = 4;
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
one_element = tensorflow.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(transform).batch(row*col)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
        plt.savefig("/kaggle/working/4.png")
    plt.show()
    break

In [ ]:
train_dataset = load_dataset(TRAINING_FILENAMES, labeled = True)
valid_dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=True)

In [ ]:
test_ds = get_test_dataset(ordered=True) 
test_images_ds = test_ds.map(lambda image, idnum: image)

## Learning Rate Scheduler 1

0 ... 10 epochs

In [ ]:
def scheduler1(epoch):
    if epoch < 4:
        return 0.0005
    elif epoch < 6:
        return 0.0001
    elif epoch < 8:
        return 0.00005
    else:
        return 0.00001

lr_callback1 = tensorflow.keras.callbacks.LearningRateScheduler(scheduler1)

# Learning Rate Scheduler 2

10 ... 20 epochs

In [ ]:
def scheduler2(epoch):
    if epoch < 4:
        return 0.000005
    elif epoch < 6:
        return 0.000001
    elif epoch < 8:
        return 0.0000005
    else:
        return 0.0000001


lr_callback2 = tensorflow.keras.callbacks.LearningRateScheduler(scheduler2)

## Learning Rate Scheduler 3

20 ... 30 epochs

In [ ]:
def scheduler3(epoch):
    if epoch < 4:
        return 0.00000005
    elif epoch < 6:
        return 0.00000001
    elif epoch < 8:
        return 0.000000005
    else:
        return 0.000000001


lr_callback3 = tensorflow.keras.callbacks.LearningRateScheduler(scheduler3)


**********************


In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(15,15), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title(title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
def display_confusion_matrix(cmat, score, precision, recall, titlestring):
    plt.figure(figsize=(45,45))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 18})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 18})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    if score is not None:
        titlestring += '\n f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\n precision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\n recall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 30, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

In [ ]:
cmdataset = get_validation_dataset(load_dataset(VALIDATION_FILENAMES, labeled = True, ordered = True)) 
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()
cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() 



********************************

********************************


# Model 1 based on EfficientNet B7 with imagenet weights

* Un-freeze the last 64 layers
* Train 10 epochs 


In [ ]:
with strategy.scope():
    #B7_base = EfficientNetB7(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B7_base = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B7_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 64 layers
    for layer in B7_base.layers:
        if layer == B7_base.layers[-64]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
    model1 = tensorflow.keras.Sequential([
        B7_base,
        tensorflow.keras.layers.GlobalAveragePooling2D(),
        #tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Dropout(0.15),
        #tensorflow.keras.layers.Dense(256, activation = 'relu'),
        #tensorflow.keras.layers.Dropout(0.1),
        tensorflow.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
model1.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model1.summary()

In [ ]:
history1 = model1.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback1], validation_data=get_validation_dataset(valid_dataset))


In [ ]:
model1.save_weights("/kaggle/working/EfficientNetB7_10epochs.h5")

In [ ]:
display_training_curves(history1.history['loss'], history1.history['val_loss'], 'EfficientNet B7 imagenet weights model  - 64 layers - Loss', 211)
display_training_curves(history1.history['sparse_categorical_accuracy'], history1.history['val_sparse_categorical_accuracy'], 'EfficientNet B7 imagenet weights model  - 64 layers - Accuracy', 212)



*********************


* Un-freeze the last 128 layers
* Train 10 epochs (20 epochs in total)


In [ ]:
with strategy.scope():

    B7_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 128 layers
    for layer in B7_base.layers:
        if layer == B7_base.layers[-128]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model1.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model1.summary()

In [ ]:
history1 = model1.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback2], validation_data=get_validation_dataset(valid_dataset))


In [ ]:
 model1.save_weights("/kaggle/working/EfficientNetB7_20epochs.h5")

In [ ]:
display_training_curves(history1.history['loss'], history1.history['val_loss'], 'EfficientNet B7 imagenet weights model  - 128 layers - Loss', 211)
display_training_curves(history1.history['sparse_categorical_accuracy'], history1.history['val_sparse_categorical_accuracy'], 'EfficientNet B7 imagenet weights model  - 128 layers - Accuracy', 212)

*********************


* Un-freeze the last 256 layers
* Train 10 epochs (30 epochs in total)


In [ ]:
with strategy.scope():

    B7_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 256 layers
    for layer in B7_base.layers:
        if layer == B7_base.layers[-256]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model1.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model1.summary()

In [ ]:
history1 = model1.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback3], validation_data=get_validation_dataset(valid_dataset))


In [ ]:
model1.save_weights("/kaggle/working/EfficientNetB7_30epochs.h5")

In [ ]:
display_training_curves(history1.history['loss'], history1.history['val_loss'], 'EfficientNet B7 imagenet weights model  - 256 layers - Loss', 211)
display_training_curves(history1.history['sparse_categorical_accuracy'], history1.history['val_sparse_categorical_accuracy'], 'EfficientNet B7 imagenet weights model  - 256 layers - Accuracy', 212)

In [ ]:

cm_probabilities = model1.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T 

titlestring = "EfficientNet B7 with imagenet weights"

display_confusion_matrix(cmat, score, precision, recall, titlestring)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))


In [ ]:
probabilities1 = model1.predict(test_images_ds)
predictions1 = np.argmax(probabilities1, axis=-1)

test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 

np.savetxt('submissionEfficientNetB7.csv', np.rec.fromarrays([test_ids, predictions1]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')



********************************

********************************



# Model 2 based on EfficientNet B6 with imagenet weights

* Un-freeze the last 64 layers
* Train 10 epochs 


In [ ]:
with strategy.scope():
    #B6_base = EfficientNetB6(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B6_base = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B6_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 64 layers
    for layer in B6_base.layers:
        if layer == B6_base.layers[-64]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
    model2 = tensorflow.keras.Sequential([
        B6_base,
        tensorflow.keras.layers.GlobalAveragePooling2D(),
        #tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Dropout(0.15),
        #tensorflow.keras.layers.Dense(256, activation = 'relu'),
        #tensorflow.keras.layers.Dropout(0.1),        
        tensorflow.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
model2.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model2.summary()

In [ ]:
history2 = model2.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback1], validation_data=get_validation_dataset(valid_dataset))


In [ ]:
model2.save_weights("/kaggle/working/EfficientNetB6_10epochs.h5")

In [ ]:
display_training_curves(history2.history['loss'], history2.history['val_loss'], 'EfficientNet B6 with imagenet weights model - 64 layers - Loss', 211)
display_training_curves(history2.history['sparse_categorical_accuracy'], history2.history['val_sparse_categorical_accuracy'], 'EfficientNet B6 with imagenet weights model  - 64 layers - Accuracy', 212)

*********************


* Un-freeze the last 128 layers
* Train 10 epochs (20 in total)


In [ ]:
with strategy.scope():

    B6_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 128 layers
    for layer in B6_base.layers:
        if layer == B6_base.layers[-128]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model2.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model2.summary()

In [ ]:
history2 = model2.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback2], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model2.save_weights("/kaggle/working/EfficientNetB6_20epochs.h5")

In [ ]:
display_training_curves(history2.history['loss'], history2.history['val_loss'], 'EfficientNet B6 with imagenet weights model - 128 layers - Loss', 211)
display_training_curves(history2.history['sparse_categorical_accuracy'], history2.history['val_sparse_categorical_accuracy'], 'EfficientNet B6 with imagenet weights model  - 128 layers - Accuracy', 212)

 
*********************


* Un-freeze the last 256 layers
* Train 10 epochs (30 in total)


In [ ]:
with strategy.scope():

    B6_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 256 layers
    for layer in B6_base.layers:
        if layer == B6_base.layers[-256]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model2.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model2.summary()

In [ ]:
history2 = model2.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback3], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model2.save_weights("/kaggle/working/EfficientNetB6_30epochs.h5")

In [ ]:
display_training_curves(history2.history['loss'], history2.history['val_loss'], 'EfficientNet B6 with imagenet weights model - 256 layers - Loss', 211)
display_training_curves(history2.history['sparse_categorical_accuracy'], history2.history['val_sparse_categorical_accuracy'], 'EfficientNet B6 with imagenet weights model  - 256 layers - Accuracy', 212)

In [ ]:
cm_probabilities = model2.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T 

titlestring = "EfficientNet B6 with imagenet weights"
display_confusion_matrix(cmat, score, precision, recall, titlestring)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
probabilities2 = model2.predict(test_images_ds)
predictions2 = np.argmax(probabilities2, axis=-1)

test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 

np.savetxt('submissionEfficientNetB6.csv', np.rec.fromarrays([test_ids, predictions2]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')


********************************

********************************


# Model 3 based on EfficientNet B5 with imagenet weights

* Un-freeze the last 64 layers
* Train 10 epochs 


In [ ]:
with strategy.scope():
    #B5_base = EfficientNetB5(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B5_base = efn.EfficientNetB5(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B5_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 64 layers
    for layer in B5_base.layers:
        if layer == B5_base.layers[-64]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    model3 = tensorflow.keras.Sequential([
        B5_base,
        tensorflow.keras.layers.GlobalAveragePooling2D(),
        #tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Dropout(0.15),
        #tensorflow.keras.layers.Dense(256, activation = 'relu'),
        #tensorflow.keras.layers.Dropout(0.1),
        tensorflow.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
model3.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model3.summary()

In [ ]:
history3 = model3.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback1], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model3.save_weights("/kaggle/working/EfficientNetB5_10epochs.h5")

In [ ]:
display_training_curves(history3.history['loss'], history3.history['val_loss'], 'EfficientNet B5 with imagenet weights model  - 64 layers - Loss', 211)
display_training_curves(history3.history['sparse_categorical_accuracy'], history3.history['val_sparse_categorical_accuracy'], 'EfficientNet B5 with imagenet weights model  -  64 layers - Accuracy', 212)



*********************


* Un-freeze the last 128 layers
* Train 10 epochs (20 in total)


In [ ]:
with strategy.scope():

    B5_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 128 layers
    for layer in B5_base.layers:
        if layer == B5_base.layers[-128]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model3.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model3.summary()

In [ ]:
history3 = model3.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback2], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model3.save_weights("/kaggle/working/EfficientNetB5_20epochs.h5")

In [ ]:
display_training_curves(history3.history['loss'], history3.history['val_loss'], 'EfficientNet B5 with imagenet weights model  -  128 layers - Loss', 211)
display_training_curves(history3.history['sparse_categorical_accuracy'], history3.history['val_sparse_categorical_accuracy'], 'EfficientNet B5 with imagenet weights model  -  128 layers - Accuracy', 212)



*********************


* Un-freeze the last 256 layers
* Train 10 epochs (30 in total)


In [ ]:
with strategy.scope():

    B5_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 256 layers
    for layer in B5_base.layers:
        if layer == B5_base.layers[-256]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model3.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model3.summary()

In [ ]:
history3 = model3.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback3], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model3.save_weights("/kaggle/working/EfficientNetB5_30epochs.h5")

In [ ]:
display_training_curves(history3.history['loss'], history3.history['val_loss'], 'EfficientNet B5 with imagenet weights model  -  256 layers - Loss', 211)
display_training_curves(history3.history['sparse_categorical_accuracy'], history3.history['val_sparse_categorical_accuracy'], 'EfficientNet B5 with imagenet weights model  -  256 layers - Accuracy', 212)

In [ ]:
cm_probabilities = model3.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T 

titlestring = "EfficientNet B5 with imagenet weights"
display_confusion_matrix(cmat, score, precision, recall, titlestring)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
probabilities3 = model3.predict(test_images_ds)
predictions3 = np.argmax(probabilities3, axis=-1)

test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 

np.savetxt('submissionEfficientNetB5.csv', np.rec.fromarrays([test_ids, predictions3]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')


********************************

********************************


# Model 4 based on EfficientNet B4 with imagenet weights

* Un-freeze the last 64 layers
* Train 10 epochs 


In [ ]:
with strategy.scope():
    #B4_base = EfficientNetB4(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B4_base = efn.EfficientNetB4(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    B4_base.trainable = True 
    
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 64 layers
    for layer in B4_base.layers:
        if layer == B4_base.layers[-64]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
    model4 = tensorflow.keras.Sequential([
        B4_base,
        tensorflow.keras.layers.GlobalAveragePooling2D(),
        #tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Dropout(0.15),
        #tensorflow.keras.layers.Dense(256, activation = 'relu'),
        #tensorflow.keras.layers.Dropout(0.1),
        tensorflow.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
model4.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model4.summary()

In [ ]:
history4 = model4.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback1], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model4.save_weights("/kaggle/working/EfficientNetB4_10epochs.h5")

In [ ]:
display_training_curves(history4.history['loss'], history4.history['val_loss'], 'EfficientNet B4 with imagenet weights model  -  64 layers - Loss', 211)
display_training_curves(history4.history['sparse_categorical_accuracy'], history4.history['val_sparse_categorical_accuracy'], 'EfficientNet B4 with imagenet weights model  -  64 layers - Accuracy', 212)

*********************


* Un-freeze the last 128 layers
* Train 10 epochs (20 epochs in total)


In [ ]:
with strategy.scope():

    B4_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 128 layers
    for layer in B4_base.layers:
        if layer == B4_base.layers[-128]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model4.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model4.summary()

In [ ]:
history4 = model4.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback2], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model4.save_weights("/kaggle/working/EfficientNetB4_20epochs.h5")

In [ ]:
display_training_curves(history4.history['loss'], history4.history['val_loss'], 'EfficientNet B4 with imagenet weights model  - 128 layers - Loss', 211)
display_training_curves(history4.history['sparse_categorical_accuracy'], history4.history['val_sparse_categorical_accuracy'], 'EfficientNet B4 with imagenet weights model  - 128 layers - Accuracy', 212)



*********************


* Un-freeze the last 256 layers
* Train 10 epochs (30 in total)


In [ ]:
with strategy.scope():

    B4_base.trainable = True 
    
    # Freeze all layers 
    set_trainable = False
    
    # Un-freeze the last 256 layers
    for layer in B4_base.layers:
        if layer == B4_base.layers[-256]: 
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    
        
model4.compile(
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00000005, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model4.summary()

In [ ]:
history4 = model4.fit(get_training_dataset(train_dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=10, callbacks = [lr_callback3], validation_data=get_validation_dataset(valid_dataset))

In [ ]:
model4.save_weights("/kaggle/working/EfficientNetB4_30epochs.h5")

In [ ]:
display_training_curves(history4.history['loss'], history4.history['val_loss'], 'EfficientNet B4 with imagenet weights model  - 256 layers - Loss', 211)
display_training_curves(history4.history['sparse_categorical_accuracy'], history4.history['val_sparse_categorical_accuracy'], 'EfficientNet B4 with imagenet weights model  - 256 layers - Accuracy', 212)

In [ ]:
cm_probabilities = model4.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T 

titlestring = "EfficientNet B4 with imagenet weights"
display_confusion_matrix(cmat, score, precision, recall, titlestring)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
probabilities4 = model4.predict(test_images_ds)
predictions4 = np.argmax(probabilities4, axis=-1)

test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 

np.savetxt('submissionEfficientNetB4.csv', np.rec.fromarrays([test_ids, predictions4]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')


*********************************



*********************************




# Make predictions using weighted ensemble

In [ ]:
probabilities = 0.6 * probabilities1 + 0.2 * probabilities2 + 0.14 * probabilities3 + 0.06 * probabilities4

predictions = np.argmax(probabilities, axis=-1)

test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 


## Create submission

In [ ]:
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')




*********************************



***************************



### Notebooks


https://www.kaggle.com/sgladysh/flowers-tpu-efficientnet-b7-b6-b5-b4-noisy-student


https://www.kaggle.com/sgladysh/flowers-tpu-efficientnet-b7-b6-b5-b4-imagenet


https://www.kaggle.com/sgladysh/flowers-tpu-efficientnet-densenet-resnet-inception



************************

